## Evaluate transfer learning on Turnedtable Watertank (Dataset 2)  using an SVM Classifier: Self-supervised approach 

In [1]:
import tensorflow as tf
import h5py
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import sys
import os
sys.path.append('../../../')
sys.path.append('../../')

### Configure Tensorflow for GPU device

In [2]:
tf.config.experimental_run_functions_eagerly(True)
print("[INFO] Tensorflow Version:", tf.__version__)

if tf.config.list_physical_devices("GPU") and tf.test.is_built_with_cuda():
    print("[INFO] Tensorflow built with CUDA")
    print("[INFO] Number GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
    print("[INFO] List of GPU devices:", tf.config.list_physical_devices("GPU"))
    physical_devices = tf.config.list_physical_devices("GPU")
    # tf.config.experimental.set_memory_growth(physical_devices[0], True)
    for gpu in physical_devices:
        tf.config.experimental.set_memory_growth(gpu, True)

else:
    print("[ERROR] GPU not detected, make sure tensorflow-gpu is installed and that GPU is recognized")
    exit()

[INFO] Tensorflow Version: 2.2.0
[INFO] Tensorflow built with CUDA
[INFO] Number GPUs Available:  1
[INFO] List of GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Define utilities

In [3]:
# utilities
def flatten(x):
    return x.reshape((x.shape[0], -1))

def classSampling(X, y, samplesPerClass, numberOfClasses):
    X_ret = np.zeros((samplesPerClass * numberOfClasses, X.shape[1]), dtype = np.float32)
    y_ret = np.zeros((samplesPerClass * numberOfClasses), dtype = np.uint8)
    count = 0

    for classIdx in range(numberOfClasses):
        indices = np.where(y == classIdx)[0]

        #if len(indices) < samplesPerClass:
        #    raise IndexError("Not enough samples for class {} to produce {} samples per class. Only {} class samples available".format(classIdx, samplesPerClass, len(indices)))

        doResample = len(indices) < samplesPerClass

        chosenIndices = np.random.choice(indices, samplesPerClass, replace = doResample)

        for ci in chosenIndices:
            X_ret[count] = X[ci]
            y_ret[count] = y[ci]

            count += 1

    return X_ret, y_ret

### Define dataset and dataloader

In [4]:
class SonarTurnedTableSupervised(object):
    def __init__(self, file_path):
        self.file_path = file_path

    def _normalize_images(self, images):
        """
        Normalize sonar images by 1/255.
        """
        return [element/255.0 for element in images]

    def get_sonar_data(self):
        """
        Reads from HDF5 file containing sonar data (resized to fix dims).
        Returns list of np arrays containing image data.
        """

        print("[INFO] Retrieving Sonar Turned Table Supervised Data")

        with h5py.File(self.file_path, "r") as f:
            # list all groups
            print("hdf5 dataset keys: %s" % f.keys())

            # get images and labels
            x_train = f["x_train"][...].astype(np.float32)
            y_train = f["y_train"][...]

            x_test = f["x_test"][...].astype(np.float32)
            y_test = f["y_test"][...]

            _, x_val, _, y_val = train_test_split(x_test, y_test, train_size=0.5)

            print("[INFO] Data dimensions")
            print("Train", len(x_train))
            print("Val", len(x_val))
            print("Test", len(x_test))

            # matias normalization
            # multiply by 255 because hdf5 file comes as 1/255
            x_train *= 255.0
            x_val *= 255.0
            x_test *= 255.0

            x_train -= 84.51
            x_val -= 84.51
            x_test  -= 84.51

        return (x_train, y_train), (x_val, y_val), (x_test, y_test)
    
def load_sonar_turnedtable_supervised(file_path):
    """
    Loads test data from turnedtable dataset.
    """
    print()
    print("[INFO] Loading Tenorflow dataset")

    dataset_object = SonarTurnedTableSupervised(file_path)

    # Read data
    (x_train, y_train), (x_val, y_val), (x_test, y_test) = dataset_object.get_sonar_data()

    # Train data
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(len(x_train))
    train_dataset = train_dataset.prefetch(25)

    # Validation data
    # val_dataset = tf.data.Dataset.from_tensor_slices((x_val, labels_val))
    # val_dataset = val_dataset.shuffle(buffer_size=len(x_val)).batch(batch_size)
    # val_dataset = val_dataset.prefetch(25)

    # Test data
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    test_dataset = test_dataset.shuffle(buffer_size=len(x_test)).batch(len(x_test)) # feed full test set
    test_dataset = test_dataset.prefetch(25)

    print()
    print("[INFO] Tensorflow data dimensions")
    # print(train_dataset)
    # print(val_dataset)
    print(test_dataset)

    # return train_dataset, val_dataset, test_dataset
    return train_dataset, test_dataset

###  Load Pretrained Model

In [5]:
from architectures.minixception import MiniXception

PRETRAINED_NUM_CLASSES = 11 # 11 supervised, 4 self-supervised
input_shape = [96, 96, 1]

model_name = "minixception"
pretraining_mode = "supervised_learning"
# layers = ["add_19", "add_18", "conv2d_35"]
layers = ["add_3", "add_2", "conv2d_6"]
model = MiniXception(input_shape, PRETRAINED_NUM_CLASSES)

model.summary()

Model: "MINI-XCEPTION"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image (InputLayer)              [(None, 96, 96, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 96, 96, 8)    72          image[0][0]                      
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 96, 96, 8)    32          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 96, 96, 8)    0           batch_normalization[0][0]        
______________________________________________________________________________________

In [6]:
# check weights BEFORE loading pretrained model (second conv layer)
print(model.layers[1].get_weights()[0])

[[[[-0.22656749  0.11373696  0.08570296  0.03367659  0.17713797
     0.24255043  0.1617583  -0.20520118]]

  [[ 0.09512931 -0.16254996 -0.06198053  0.10725272 -0.21955788
     0.15142456  0.00743008  0.12286189]]

  [[ 0.20801693 -0.20456883  0.03464344 -0.05148812  0.1618973
    -0.07576503 -0.2351779  -0.15021041]]]


 [[[ 0.00722271  0.1193338  -0.13276738 -0.07977325 -0.02457066
     0.05066136  0.1505394   0.10930398]]

  [[-0.04570045  0.02812484  0.07715216  0.08815533 -0.20191532
     0.00083506 -0.09768918  0.11116859]]

  [[-0.12322819  0.19105622 -0.12652723 -0.08467375 -0.03722973
    -0.05861115 -0.00185338 -0.20767827]]]


 [[[-0.24994238 -0.10566071  0.0489085   0.14529908 -0.0817157
    -0.08794859 -0.0565844  -0.07884876]]

  [[-0.12576082 -0.14211488 -0.18695281  0.26149952  0.21013296
    -0.23714672 -0.01795456 -0.13512176]]

  [[-0.20738855  0.18661767  0.18953139  0.08667177  0.13910168
    -0.15670136 -0.16016592 -0.11844884]]]]


In [7]:
# path to pretrained model checkpoint
pretrained_checkpoint_prefix = os.path.join("../../../pretraining/results/" + pretraining_mode + "/checkpoints/sonar1/" + model_name + "/batch_size_128/96x96_substract_mean_online_aug_width_16")

# optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.5, epsilon=1)

# define pretrained checkpoint model
checkpoint_pretrained = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optimizer, model=model) # 4 ssl, 11 sl
manager_pretrained = tf.train.CheckpointManager(checkpoint_pretrained, pretrained_checkpoint_prefix, max_to_keep=3)

# restore model weights
checkpoint_pretrained.restore(manager_pretrained.latest_checkpoint)

if manager_pretrained.latest_checkpoint:
    print("[INFO] Pretrained checkpoint restored correctly: {}".format(manager_pretrained.latest_checkpoint))
else:
    print("[INFO] Could not restore pretrained checkpoint correctly, make sure path to pre-trained folder is correct.")

[INFO] Pretrained checkpoint restored correctly: ../../../pretraining/results/supervised_learning/checkpoints/sonar1/minixception/batch_size_128/96x96_substract_mean_online_aug_width_16/ckpt-21


In [8]:
# check weights AFTER loading pretrained model (second conv layer)
print(model.layers[1].get_weights()[0])

[[[[-0.1969977  -0.16114475 -0.05452951 -0.01648649 -0.04397418
     0.16738395  0.05095789  0.00464345]]

  [[ 0.25952905 -0.20669673 -0.1146349  -0.15266207 -0.206317
    -0.09935854 -0.1080291   0.23363659]]

  [[ 0.22118595 -0.0129049  -0.19327141  0.13474298 -0.00441496
    -0.20215212 -0.1692035   0.04673611]]]


 [[[ 0.08820812  0.0537549  -0.02675858 -0.1524948   0.15513435
     0.15246893 -0.07941455  0.23962311]]

  [[-0.02320163 -0.2597428  -0.01100731 -0.22715679  0.26726216
     0.03903156 -0.03175402 -0.2116491 ]]

  [[ 0.07139531 -0.19967245 -0.11045456  0.14042066  0.10249978
     0.08826296  0.12672648 -0.16266125]]]


 [[[-0.06918748  0.08518618 -0.03868058  0.05516814 -0.21323991
     0.16145857 -0.05402087  0.28015283]]

  [[-0.06179183 -0.2115543   0.17678131  0.03347379  0.16450408
     0.093099    0.22170022 -0.05321667]]

  [[-0.1072505   0.1445197   0.26691496 -0.16406988 -0.16968
     0.11769076  0.10293937 -0.2159432 ]]]]


### Define models up to intermediate layers


In [9]:
print("Intermediate layers from model:", layers)

Intermediate layers from model: ['add_3', 'add_2', 'conv2d_6']


In [10]:
# https://stackoverflow.com/questions/63297838/how-can-i-obtain-the-output-of-an-intermediate-layer-feature-extraction
add_3 = tf.keras.Model(inputs=model.get_layer("image").output, 
                                     outputs=model.get_layer(layers[0]).output)

add_2 = tf.keras.Model(inputs=model.get_layer("image").output, 
                                     outputs=model.get_layer(layers[1]).output)

conv2d_6 = tf.keras.Model(inputs=model.get_layer("image").output, 
                                     outputs=model.get_layer(layers[2]).output)


### Generate vector embeddings for train and test data (up to n-th layer)

In [11]:
# define tensorflow dataset
data_dir = "../../../../../../datasets/sonar_turntable_dataset_2/marine-debris-turntable-classification-object_classes-platform-96x96.hdf5"
train_dataset, test_dataset = load_sonar_turnedtable_supervised(data_dir)

# load tensorflow tensors individually
x_train, y_train = next(iter(train_dataset))
x_test, y_test = next(iter(test_dataset))


[INFO] Loading Tenorflow dataset
[INFO] Retrieving Sonar Turned Table Supervised Data
hdf5 dataset keys: <KeysViewHDF5 ['class_names', 'x_test', 'x_train', 'y_test', 'y_train']>
[INFO] Data dimensions
Train 1505
Val 323
Test 645

[INFO] Tensorflow data dimensions
<PrefetchDataset shapes: ((None, 96, 96, 1), (None,)), types: (tf.float32, tf.int64)>


In [12]:
# perform a forward pass to generate embeddings (both train and test data) (for each n-th layer)

# NOTE: when doing this all tensors are loaded into memory, this saturates NVIDIA memory (nvidia-smi)
x_train_add_3 = add_3([x_train], training=False)
x_train_add_2 = add_2([x_train], training=False)
x_train_conv2d_6 = conv2d_6([x_train], training=False)

x_test_add_3 = add_3([x_test], training=False)
x_test_add_2 = add_2([x_test], training=False)
x_test_conv2d_6 = conv2d_6([x_test], training=False)

### Transfer Learning setup: classification with subsamples per object class (few shot learning)

In [13]:
# transfer learning params
# SAMPLES_PER_CLASS = [1, 5, 10, 20, 30, 40, 50] # NOTE: taking more samples per class since it is 88 for 50
# SAMPLES_PER_CLASS = [10, 20, 30, 40, 50, 70, 90, 110, 130, 150, len(x_test)]
SAMPLES_PER_CLASS = [10, 20, 30, 40, 50, 80, 110, 140, 170, 200]
TRIALS = 10

NUM_CLASSES_WATERTANK = 11
NUM_CLASSES_TURNEDTABLE = 12

In [14]:
# Flatten train & test data for SVM
x_train_add_3 = flatten(x_train_add_3.numpy())
x_train_add_2 = flatten(x_train_add_2.numpy())
x_train_conv2d_6 = flatten(x_train_conv2d_6.numpy())

x_test_add_3 = flatten(x_test_add_3.numpy())
x_test_add_2 = flatten(x_test_add_2.numpy())
x_test_conv2d_6 = flatten(x_test_conv2d_6.numpy())

In [15]:
# these two are not modified (only x_test, x_train)
print(y_train.numpy().shape)
print(y_test.numpy().shape)

y_train = y_train.numpy() # convert from tf tensor --> numpy
y_test = y_test.numpy()

(1505,)
(645,)


### Run svm tl evaluation with spc for each n-th layer

In [16]:
def train_svm_with_spc(x_train, y_train, x_test, y_test):
    """
    Takes embeddings from pretrained model and evaluates transfer learning 
    with few samples per class.
    """
    # NOTE: svm takes original labels (not one-hot encoding)
    for spc in SAMPLES_PER_CLASS:
        accuracies = []

        for i in range(TRIALS):
            x_sample, y_sample = classSampling(x_train, y_train, spc, NUM_CLASSES_TURNEDTABLE)

            svm = SVC(C=1.0, decision_function_shape = 'ovo', kernel="linear")
            svm.fit(x_sample, y_sample)

            train_acc = svm.score(x_sample, y_sample)
            test_acc = svm.score(x_test, y_test)

            print("SPC {} Train Accuracy: {:.3f}".format(spc, train_acc))
            print("SPC {} Test Accuracy: {:.3f}".format(spc, test_acc))
            print()

            accuracies.append(test_acc)

        mean_acc = np.mean(accuracies)
        std_acc = np.std(accuracies)

        mean_acc = round(100 * mean_acc, 3)
        std_acc = round(100 * std_acc, 3)

        print("After {} trials - Test Accuracy is {} +- {}".format(TRIALS, mean_acc, std_acc ))
        print("------------------------------------------------------------------------------")
        print()

In [17]:
train_svm_with_spc(x_train_add_3, y_train, x_test_add_3, y_test)

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.712

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.665

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.673

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.713

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.696

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.766

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.778

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.722

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.705

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.702

After 10 trials - Test Accuracy is 71.333 +- 3.393
------------------------------------------------------------------------------

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.820

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.842

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.826

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.833

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.820


In [18]:
train_svm_with_spc(x_train_add_2, y_train, x_test_add_2, y_test)

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.699

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.712

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.712

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.671

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.670

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.730

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.733

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.685

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.729

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.693

After 10 trials - Test Accuracy is 70.341 +- 2.237
------------------------------------------------------------------------------

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.797

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.803

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.834

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.802

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.829


In [19]:
train_svm_with_spc(x_train_conv2d_6, y_train, x_test_conv2d_6, y_test)

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.659

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.640

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.583

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.665

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.687

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.688

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.634

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.592

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.626

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.650

After 10 trials - Test Accuracy is 64.248 +- 3.363
------------------------------------------------------------------------------

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.738

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.738

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.744

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.763

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.741


In [21]:
x_train_conv2d_6.shape

(1505, 396)

In [22]:
x_train_add_2.shape

(1505, 9216)

In [23]:
x_train_add_3.shape

(1505, 4608)